- Convert Dataset (jsonl) -> fine-tune dolphin-mistal-7b -> quantize -> create ollama Modelfile -> push .gguf (HF)

TODO:
- Convert .ipynb to py
  - dataset.py | train.py
- Create config.yaml
  - Base Model + Dataset
  - Training parameters
  - Quantization Methods
- Check Discord implementation

In [1]:
from huggingface_hub import HfApi
from datasets import load_dataset
import subprocess
import json
import os

def convert_to_jsonl(input_directory, output_filename):
    os.makedirs(os.path.dirname(output_filename), exist_ok=True)
    
    all_chats = []  # To hold all chats before writing to JSONL

    for input_filename in os.listdir(input_directory):
        if input_filename.endswith('.txt'):
            bot_name = input_filename[:-4]  # Remove '.txt' from filename to get bot_name
            file_path = os.path.join(input_directory, input_filename)
                
            with open(file_path, 'r', encoding='utf-8') as file:
                conversation_blocks = file.read().split('\n\n')
                
            for block in conversation_blocks:
                lines = block.split('\n')
                chat = [{"role": "system", "content": bot_name}]
                for line in lines:
                    if line.startswith("HUMAN:"):
                        chat.append({"role": "user", "content": line.replace("HUMAN:", "").strip()})
                    elif line.startswith("RESPONSE:"):
                        chat.append({"role": "assistant", "content": line.replace("RESPONSE:", "").strip()})

                if len(chat) > 1:  # Ensure there's more than just the system line
                    all_chats.append({"chat": chat})
    
    # Writing all chats to a single JSONL file
    with open(output_filename, 'w', encoding='utf-8') as jsonl_file:
        for chat in all_chats:
            json.dump(chat, jsonl_file)
            jsonl_file.write('\n')

# Configuration
base_dir = 'F:/discollama'
repo_dir = 'datasets/TRACHI'
input_directory = os.path.join(base_dir, 'datasets', 'raw')
output_filename = os.path.join(base_dir, repo_dir, 'train.jsonl')

# Convert txt files to a single jsonl file
convert_to_jsonl(input_directory, output_filename)

subprocess.run(["huggingface-cli", "login", os.getenv('HUGGINGFACE_TOKEN')])
dataset = load_dataset(repo_dir)
dataset.push_to_hub("norygano/TRACHI")

Generating train split: 0 examples [00:00, ? examples/s]